In [13]:
import io
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression 
from keras.utils import to_categorical

###########################################################################
#generate training/test data
########################################################################### 
bragging_file = io.open("cleantweets/positive.txt",'r',encoding="utf-8") 
bitching_file = io.open("cleantweets/negative.txt",'r',encoding="utf-8") 


bragging_tweets = []
bitching_tweets = []

for _ in range(250000):
    bragging_tweets.append([bragging_file.readline().strip("\n"),0])
    bitching_tweets.append([bitching_file.readline().strip("\n"),1])
    
tweets = bragging_tweets+bitching_tweets
np.random.seed(42)
np.random.shuffle(tweets)


tweets_train = np.array(tweets[:300000])
tweets_test = np.array(tweets[300000:])

X_train = tweets_train[:,0]
Y_train = to_categorical(np.array([[int(i)] for i in tweets_train[:,1]],dtype=np.int))
X_test = tweets_test[:,0]
Y_test = to_categorical(np.array([[int(i)] for i in tweets_test[:,1]],dtype=np.int))

test_tweets = ["Fuck you mom","I love you mom","I hate homework","I enjoy homework", "Life sucks","Life is good",
               "Jesus loves me","Put a ring on it","Mercedes Benz: $20,000",
                "The decision on Sergeant Bergdahl is a complete and total disgrace to our Country and to our Military.",
                "Does anybody really believe that a reporter, who nobody ever heard of, went to his mailbox and found my tax returns? @NBCNews FAKE NEWS!",
                "After Turkey call I will be heading over to Trump National Golf Club, Jupiter, to play golf (quickly) with Tiger Woods and Dustin Johnson. Then back to Mar-a-Lago for talks on bringing even more jobs and companies back to the USA!",
                "“The President has accomplished some absolutely historic things during this past year.” Thank you Charlie Kirk of Turning Points USA. Sadly, the Fake Mainstream Media will NEVER talk about our accomplishments in their end of year reviews. We are compiling a long & beautiful list.",
                "Crazy Joe Scarborough and dumb as a rock Mika are not bad people, but their low rated show is dominated by their NBC bosses. Too bad!",
                "Robert Pattinson should not take back Kristen Stewart. She cheated on him like a dog & will do it again--just watch. He can do much better!"]

#########################################################################
#bag of words
#########################################################################

vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)
test_tweets_counts = vectorizer.transform(test_tweets)




#Children 's #Day to u all.....may #GOD #bless and #protect our #children in the name of #JESUS #AMEN from... http:// fb.me/4ifqcgWnA


In [15]:

import keras.optimizers
from keras.models import Sequential 
from keras.layers import Dense, Activation 

model = Sequential() 
model.add( Dense(units=2, activation="softmax", input_dim= X_train_counts.shape[1], use_bias=False) )


In [16]:
sgd = keras.optimizers.SGD(lr=0.0001*400)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:


history = model.fit(X_train_counts, Y_train, epochs=20,validation_split = 0.2)
model.save_weights('Simple_Logistic_Regression.h5')

Epoch 1/1


InternalError: GPU sync failed

In [24]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) +1)

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [26]:
model.load_weights('Simple_Logistic_Regression.h5')
print (model.evaluate(X_test_counts, Y_test))


In [53]:
predictions = predict(model.predict(test_tweets_counts))
for i in range(len(predictions)):
    print(test_tweets[i],"\n",predictions[i])

[ 1.  0.  1. ...,  0.  1.  0.]
(149400,)
1.0


149400/149400 [==============================] - 48s 323us/step
[0.32724456065629859, 0.87233601070631306]


Accuracy with logistic regression:.... 0.0


/Users/hamzakhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


[[ 0.57011467]
 [ 0.65407145]]
